## Данный чат бот поможет выбрать блюдо, которое вы можете приготовить в зависимости от ваших предпочтений

Чат бот подбирает подходящие пользователю рецепты на основе его ответов на серию вопросов. Основная особенность бота в том, что он использвует не дерево, а теги. Т.е. вместо очень большого кол-ва путей с развилкой на каждом выборе пользователь выбирает так называемые теги на каждом шаге. У каждого рецепта также существуют теги. И в конце пути бот подбирает подходящие под теги пользователя рецепты.

***Все рецепты взяты с данного [сайта](https://eda.ru/recepty)***

In [2]:
class cooking_bot(object):  # Для реализации бота я решила использовать класс
    def __init__(self):
        # Конструктор класса

        self.working = True  # Поле, которое хранит состояние работы бота
        self.on_stage = False  # Поле, которое хранит, находится ли пользователь в процессе ввода
        self.fit_recipes = []  # Список подошедших рецептов
        self.recipes_tags = {
            'Традиционные спагетти болоньезе': dict(dtime=['обед', 'ужин'], comp=['средний'], max_time=110,
                                                    ingr=['оливковое масло', 'чеснок', 'лук', 'говяжий фарш', 'сахар',
                                                          'консервированные помидоры',
                                                          'спагетти', 'сыр пармезан', 'говяжий бульон', 'сельдерей',
                                                          'красное сухое вино',
                                                          'петрушка'],
                                                    link='https://eda.ru/recepty/pasta-picca/tradicionnye-spagetti'
                                                         '-boloneze-19516'),

            'Пышные американские панкейки': dict(dtime=['завтрак'], comp=['простой'], max_time=15,
                                                 ingr=['молоко', 'куриное яйцо', 'сахар', 'разрыхлитель',
                                                       'ванильный сахар',
                                                       'пшеничная мука'],
                                                 link='https://eda.ru/recepty/zavtraki/pishnie-amerikanskie-pankejki'
                                                      '-42716'),

            'Брауни': dict(dtime=['завтрак', 'обед', 'ужин'], comp=['средний'], max_time=40,
                           ingr=['темный шоколад', 'сливочное масло', 'коричневый сахар', 'куриное яйцо',
                                 'пшеничная мука',
                                 'грецкие орехи'], link='https://eda.ru/recepty/vypechka-deserty/brauni-brownie-20955'),

            'Лазанья классическая с мясом': dict(dtime=['обед', 'ужин'], comp=['сложный'], max_time=40,
                                                 ingr=['мясной фарш', 'соус болоньезе', 'сливочное масло',
                                                       'пшеничная мука',
                                                       'оливковое масло', 'молоко', 'готовые сухие листы лазаньи',
                                                       'твердый сыр'],
                                                 link='https://eda.ru/recepty/pasta-picca/lazanja-klassicheskaja-s'
                                                      '-mjasom-31799'),

            'Спагетти карбонара': dict(dtime=['обед', 'ужин'], comp=['средний'], max_time=30,
                                       ingr=['спагетти', 'панчетта', 'твердый сыр', 'яичный желток', 'соль',
                                             'свежемолотый черный перец'],
                                       link='https://eda.ru/recepty/pasta-picca/spagetti-karbonara-81236'),

            'Картошка по-деревенски в духовке': dict(dtime=['обед', 'ужин'], comp=['средний'], max_time=40,
                                                     ingr=['картофель', 'растительное масло', 'соль',
                                                           'молотый черный перец'],
                                                     link='https://eda.ru/recepty/osnovnye-blyuda/kartoshka-po'
                                                          '-derevenski-v-duhovke-47653'),

            'Мясо по-французски с сыром': dict(dtime=['обед', 'ужин'], comp=['средний'], max_time=70,
                                               ingr=['мясо', 'репчатый лук', 'сыр', 'майонез', 'соль',
                                                     'молотый черный перец'],
                                               link='https://eda.ru/recepty/osnovnye-blyuda/mjaso-po-francuzski-s'
                                                    '-sirom-20645'),

            'Сырники из творога': dict(dtime=['завтрак'], comp=['простой'], max_time=30,
                                       ingr=['творог', 'куриное яйцо', 'пшеничная мука', 'сахар', 'подсолнечное масло'],
                                       link='https://eda.ru/recepty/zavtraki/sirniki-iz-tvoroga-18506')

        }  # Словарь рецептов с их тегами
        self.ingredients = set()
        for rec in self.recipes_tags.values():
            for ing in rec['ingr']:
                self.ingredients.add(ing)
        # Сбор уникальных ингредиентов со всех рецептов

        self.tags = {'dtime': None, 'comp': None, 'ingr': [], 'max_time': None}  # Теги, которые ввел пользователь
        self.cases = {
            'begin': {'question': 'Приступим?', 'answers': ['да', 'нет']},
            'dtime': {'question': 'На какой прием пищи вы бы хотели рецепт?',
                      'answers': ['завтрак', 'обед', 'ужин', 'не важно']},
            'comp': {'question': 'Какой сложности вы бы хотели рецепт?',
                     'answers': ['простой', 'средний', 'сложный', 'не важно']},
            'ingr': {'question': 'Какие ингредиенты вы бы хотели использовать?',
                     'answers': self.ingredients},
            'ingr_prior': {'question': 'Насколько важно наличие указанных ингредиентов?',
                           'answers': ['не важно', 'все должны быть', 'хотя бы половину', 'хотя бы четверть']},
            'max_time': {'question': 'Какой лимит по времени (в минутах)?',
                         'answers': ['<любое кол-во минут>', 'не важно']}
        }  # Словарь тегов и доступных значений

        self.body()
        # Также основная функция запускается из конструктора, чтобы можно было запустить бота просто созданием объекта

    @staticmethod
    def tr_time(mins):
        # Метод для перевода минут в часы, минуты

        return f'{mins // 60} час(-ов) {mins % 60} минут'

    def step(self, question, answers, time=False):
        # Метод для унифицированной обработки ввода пользователя

        print(question)
        print(f'Доступные варианты ответов: {answers}')
        self.on_stage = True

        while self.working and self.on_stage:  # Цикл крутится, пока полученный ответ не устроит программу
            try:  # Механизм для корректной работы с остановкой ячейки
                answer = input().strip().lower()  # Нормализация ввода пользователя
                if answer == 'стоп':  # Вариант с завершением программы словом
                    self.goodbye()
                    return False

            except KeyboardInterrupt:  # Ожидание остановки ячейки пользователем
                self.goodbye()
                return False

            else:
                flag = False  # Механизм с флагом применен из за возможности работы в с временем
                if time:
                    if answer.isdigit():  # Обработка времени на входе
                        flag = True
                        answer = int(answer)

                    if answer == 'не важно':
                        flag = True

                elif answer in answers:  # Обработка слов
                    flag = True

                if not flag:  # Ответ пользователя не устроил программу
                    print('Неподходящий ответ! Пожалуйста выберите из доступных вариантов.')
                    print(f'Доступные варианты ответов: {answers}')

                else:  # Ответ устроил и функция его возвращает
                    self.on_stage = False
                    return answer

    def step_many(self, question, answers):
        # Метод, аналогичный step, но для ввода множества значений

        print(question)
        print(f'Можете выбрать несколько вариантов ответов из следующих: {answers}')
        print('Вводите все варианты через ", ". Пример: "желтый, красный, синий"')
        self.on_stage = True

        while self.working and self.on_stage:
            try:
                answer = input().strip().lower()
                if answer == 'стоп':
                    self.goodbye()
                    return False

                answer = answer.split(', ')

            except KeyboardInterrupt:
                self.goodbye()
                return False

            else:
                miss = False
                for kind in answer:  # Ключевое отличие в данном цикле перебора тегов
                    if kind not in answers:
                        miss = True
                if miss:
                    print('Неподходящий ответ! Хотя бы один из введенных вариантов не подходит.')
                    print(f'Доступные варианты ответов: {answers}')
                else:
                    self.on_stage = False
                    return answer

    def goodbye(self):
        # Метод завершения работы бота

        self.working = False
        print('\nРабота бота завершена, спасибо!')

    @staticmethod
    def in_ok(value, spis):
        # Метод проверки соответствия тега

        if value in ['не важно'] + spis:
            return True
        else:
            return False

    @staticmethod
    def time_ok(s_t, t):
        # Метод для проверки соответствия времени

        if s_t == 'не важно' or s_t >= t:
            return True
        else:
            return False

    def fit(self):
        # Метод подбора рецептов

        fit_recipes = []
        for recipe, tags in self.recipes_tags.items():
            # Цикл бежит по всем существующим рецептам и проверяет, подходит ли каждый

            if self.in_ok(self.tags['dtime'], tags['dtime']) and self.in_ok(self.tags['comp'], tags['comp']) \
                    and self.time_ok(self.tags['max_time'], tags['max_time']):
                # Условие по приему пищи, сложности и времени приготовления
                is_in = 0
                not_in = 0

                # Далее блок условий по проверке тегов
                for ingredient in self.tags['ingr']:
                    if ingredient in tags['ingr']:
                        is_in += 1
                    else:
                        not_in += 1

                if self.tags['ingr_prior'] == 'не важно':
                    fit_recipes.append(recipe)

                elif self.tags['ingr_prior'] == 'все должны быть' and not_in == 0:
                    fit_recipes.append(recipe)

                elif self.tags['ingr_prior'] == 'хотя бы половину' and is_in / (not_in + is_in) >= 0.5:
                    fit_recipes.append(recipe)

                elif self.tags['ingr_prior'] == 'хотя бы четверть' and is_in / (not_in + is_in) >= 0.25:
                    fit_recipes.append(recipe)

        self.fit_recipes = fit_recipes  # "утверждаем" подошедшие рецепты

    def print_results(self):
        # Метод для вывода результатов

        print('\nПодходящие рецепты:')
        for rn, recipe_name in zip(range(len(self.fit_recipes)), self.fit_recipes):
            # Вывод по каждому рецепту

            recipe = self.recipes_tags[recipe_name]
            print(f'\n{rn + 1}. {recipe_name}')

            print(' Время приготовления:')
            print('  ' + self.tr_time(recipe['max_time']))

            print(' Ингредиенты:')
            print('  ' + ', '.join(recipe['ingr']))

            print(' Ссылка на рецепт:')
            print('  ' + recipe['link'])

        if not self.fit_recipes:  # Вывод в случае, если ни один рецепт не подошел
            print('К сожалению подходящих рецептов не найдено (')

    def long_story(self):  # История о Рататуе)
        ans7 = self.step('Рецепты скоро будут, а пока, хотите ли вы услышать историю о Рататуе?',
                         ['да', 'нет'])
        if ans7 == 'да':
            print('Крысёнок Реми живёт со своей семьёй на чердаке сельского домика. Главной особенностью Реми является '
                  'его чрезвычайно тонкое обоняние и чувство вкуса. Телепередача о книге лучшего повара Франции Огюста '
                  'Гюсто (его прототип — Бернар Луазо) открывает перед ним мир кулинарии. Он мечтает стать настоящим '
                  'шеф-поваром, что, естественно, не вызывает одобрения со стороны его отца Джанго — вожака стаи, '
                  'ненавидящего всё, что связано с людьми, в то время как брат Реми, Эмиль, относится к этому '
                  'нейтрально. '
                  ' Увы, неудачный кулинарный эксперимент приводит к тому, что хозяйка дома узнаёт о существовании в '
                  'своём доме крысиной семьи и прогоняет её. Реми успевает увидеть, как по телевизору передают '
                  'сообщение, '
                  'что Гюсто скончался, когда его ресторан лишили одной звезды из-за разгромной статьи критика '
                  'Антуана Эго '
                  '(вдобавок после смерти Гюсто у ресторана отняли ещё одну звезду). Преследуемый вооружённой '
                  'старушкой, '
                  'Реми попадает в водосток и теряет своих близких. Он впадает в отчаяние, но тут к нему на помощь '
                  'приходит дух Гюсто и ободряет его.')

            ans8 = self.step('Продолжаем?', ['да', 'нет'])
            if ans8 == 'да':
                print('Выбравшись наверх, он обнаруживает, что находится в Париже, возле ресторана знаменитого Гюсто. '
                      'Реми проникает на кухню ресторана. Не в силах совладать с собой, Реми спасает суп, испорченный '
                      'только что нанятым уборщиком Альфредо Лингвини. Но эта попытка едва не стоит Реми жизни: его '
                      'замечают, ловят и поручают Лингвини уничтожить крысёнка. Однако юноша и Реми находят общий язык '
                      'и с этого момента начинают работать вместе. Для удобства Лингвини зовёт своего маленького друга '
                      '«микрошефом». Смысл их совместной работы заключается в том, что Реми помогает Лингвини готовить '
                      'блюда, управляя его действиями наподобие кукловода — он сидит у него на голове и дёргает за '
                      'волосы. '
                      '\nРуками товарища Реми создаёт необычные кулинарные шедевры, чем склоняет на свою сторону '
                      'посетителей '
                      'и кулинарных критиков. При этом у Лингвини начинается роман с шефом Колетт Тату, лихой девицей, '
                      'разъезжающей на спорт-байке (та, в свою очередь, обучает его всем тонкостям кулинарии и '
                      'постепенно '
                      'проникается к воспитаннику нежными чувствами). Попутно обнаруживается, что Лингвини является '
                      'сыном '
                      'Гюсто (последний был влюблён в Ренату, покойную мать Альфредо) и по наследству должен получить '
                      'ресторан, в котором всё это время хозяйничает новый шеф-повар — итальянец Живодэр. А тому явно '
                      'не хочется расстаться с должностью…')

                ans9 = self.step('Ну что-ж, дальше?', ['да', 'нет'])
                if ans9 == 'да':
                    print('Тем не менее, Лингвини всё же становится владельцем ресторана благодаря Реми (тот выкрал у '
                          'Живодэра нужные документы, подтверждающие его право на наследство), и Живодэру приходится '
                          'уступить Альфредо своё место. Заведение снова становится популярным, как и в то время, '
                          'когда '
                          'сам Гюсто был ещё жив. В то же самое время Реми неожиданно вновь находит свою семью, '
                          'с которой '
                          'уже и не надеялся увидеться. Его братец Эмиль начинает всё чаще и чаще приводить других '
                          'крыс '
                          'к Реми за едой. Но однажды Реми из-за своей ссоры с Лингвини (крыс мешал повару на пресс-'
                          'конференции, сидя у него под поварским колпаком и периодически дёргая за волосы) ночью '
                          'приводит '
                          'всю семью в ресторан. Он угощает своих сородичей всем, что есть в кладовой ресторана. '
                          'Пришедший '
                          ' туда Лингвини, увидев ораву крыс, в гневе выгоняет их всех вместе с Реми, с которым за '
                          'несколько минут до этого хотел помириться. Живодэр, случайно узнавший о кулинарном таланте '
                          'Реми, ловит и сажает того в клетку, чтобы потом заставить работать на себя под угрозой '
                          'смерти, '
                          'но Эмиль с отцом и крысом-силачом спасают его. \nНаступает решающий день: в ресторан пришёл '
                          'самый знаменитый и суровый ресторанный критик Антуан Эго. Чтобы избежать повторного падения,'
                          ' повара должны приготовить то блюдо, которое особенно понравится гостю. Но Лингвини не в '
                          ' состоянии руководить работой на кухне без Реми, а последний, в свою очередь, не может '
                          ' допустить окончательного провала и, освободившись из плена, сразу бежит на кухню. Весь'
                          ' персонал, завидев Реми на пороге ресторана, бросается на того с тумаками, но Лингвини, '
                          ' чтобы спасти жизнь «микрошефа», раскрывает поварам свой секрет, в результате чего те в'
                          ' дурном настроении и разочаровании бросают свою работу, в том числе и Колетт (последняя '
                          ' хотела влепить Лингвини пощёчину за его ложь, однако была слишком расстроена для этого, и '
                          'с горя ушла).')

                    ans10 = self.step('Готовы к финалу?', ['да', 'нет'])
                    if ans10 == 'да':
                        print('Но тут старик Джанго, отец Реми, призывает весь свой клан на помощь сыну, несмотря даже '
                              'на отрицательное отношение к людям. Лингвини берёт на себя роль официанта (что, '
                              'впрочем, '
                              'ему удаётся гораздо лучше, чем роль повара), а Реми вместе с семьёй начинают готовить. '
                              'А '
                              'Альфредо, надев ролики, обслуживает гостей, заменяя весь штат официантов. Колетт, '
                              'проезжая '
                              'мимо книжной витрины, видит книгу Гюсто и, вспомнив его знаменитую фразу «Готовить '
                              'могут '
                              'все!», возвращается на кухню ресторана. Зайдя внутрь, она чуть не падает в обморок при '
                              'виде огромного количества крыс-«поваров», однако Реми показывает ей рецепт рататуя, '
                              'изюминка которого кроется в соусе по его рецепту, и она соглашается помочь ему в '
                              'приготовлении блюда. Результат их усердных стараний приводит критика в полный восторг '
                              '(в частности, в приготовленном Реми рататуе под бесподобным по вкусу соусом он узнаёт то'
                              ' самое простое, но вкусное овощное рагу, которое мать готовила ему в детстве). Суровый '
                              'Эго просит вызвать шеф-повара. Колетт сообщает, что это можно сделать только после '
                              'ухода '
                              'гостей. \nЛингвини и Колетт представляют взору Антуана Эго подлинного шеф-повара. '
                              'Благодаря '
                              ' его восторженному отзыву ресторан в очередной раз обретает былой престиж, однако '
                              'ненадолго. Санитарный инспектор, запертый крысами вместе с Живодэром в кладовке, а '
                              'затем отпущенный на свободу, навсегда закрывает ресторан за пренебрежение санитарными '
                              'нормами. Эго теряет доверие и должность ресторанного критика. Тем не менее, '
                              'он вкладывает '
                              'свои деньги в новое заведение Лингвини и шеф-повара Реми — пользующийся колоссальной '
                              'популярностью ресторан-бистро «Рататуй». Альфредо и Колетт окончательно стали парой, а '
                              'Реми теперь готовит вкуснейшие блюда благодаря помощи своих сородичей.\n')

        print('Спасибо за внимание)')

    def body(self):
        # Основной метод работы бота

        # Вывод предисловия
        print(
            'Добрый день! Данный бот поможет вам выбрать блюдо, которое можно приготовить с учетом ваших предпочтений '
            'и '
            'поделится рецептом для него. Регистр ответов не важен! (можно вводить и заглавные буквы). Для того, '
            'чтобы остановить работу бота, можете ввести слово "стоп" в любой момент или просто остановить ячейку.')

        ans1 = self.step(self.cases['begin']['question'], self.cases['begin']['answers'])

        # Далее унифицированный вывод вопросов и обработки ответов по очереди
        if ans1 == 'да':
            ans2 = self.step(self.cases['dtime']['question'], self.cases['dtime']['answers'])
            self.tags['dtime'] = ans2
            if not self.working:  # Условие для корректной остановки бота
                return

            ans3 = self.step(self.cases['comp']['question'], self.cases['comp']['answers'])
            self.tags['comp'] = ans3
            if not self.working:
                return

            ans4 = self.step_many(self.cases['ingr']['question'], self.cases['ingr']['answers'])
            self.tags['ingr'] = ans4
            if not self.working:
                return

            ans5 = self.step(self.cases['ingr_prior']['question'], self.cases['ingr_prior']['answers'])
            self.tags['ingr_prior'] = ans5
            if not self.working:
                return

            ans6 = self.step(self.cases['max_time']['question'], self.cases['max_time']['answers'], time=True)
            self.tags['max_time'] = ans6
            if not self.working:
                return

            self.long_story()

            self.fit()  # Подбор рецептов
            self.print_results()  # Вывод результатов

        self.goodbye()  # Окончание работы бота


cooking_bot();


Добрый день! Данный бот поможет вам выбрать блюдо, которое можно приготовить с учетом ваших предпочтений и поделится рецептом для него. Регистр ответов не важен! (можно вводить и заглавные буквы). Для того, чтобы остановить работу бота, можете ввести слово "стоп" в любой момент или просто остановить ячейку.
Приступим?
Доступные варианты ответов: ['да', 'нет']
да
На какой прием пищи вы бы хотели рецепт?
Доступные варианты ответов: ['завтрак', 'обед', 'ужин', 'не важно']
не важно
Какой сложности вы бы хотели рецепт?
Доступные варианты ответов: ['простой', 'средний', 'сложный', 'не важно']
не важно
Какие ингредиенты вы бы хотели использовать?
Можете выбрать несколько вариантов ответов из следующих: {'сливочное масло', 'сыр', 'соль', 'свежемолотый черный перец', 'лук', 'разрыхлитель', 'красное сухое вино', 'молотый черный перец', 'говяжий бульон', 'репчатый лук', 'сельдерей', 'коричневый сахар', 'растительное масло', 'мясо', 'картофель', 'творог', 'соус болоньезе', 'готовые сухие листы лаз

### Описание вещей не из курса:

1. Классы в python - `class` позволяет создать свой объект с нужными полями и методами (переменные и функции класса) <br>
    https://docs.python.org/3/tutorial/classes.html <br>
    ***В программе используется для упрощения взаимодействия всех функций и переменных***
    <br><br>
    
2. Метод `__init__` - конструктор класса, вызывается при создании объекта <br>
    https://younglinux.info/oopython/init <br><br>
    
3. `@` декораторы - в общем виде позволяют внешне модифицировать какую-либо функцию <br>
    https://wiki.python.org/moin/PythonDecorators <br><br>

4. Декоратор `@staticmethod` - позволяет использовать метод класса "отдельно" от объектов класса, по сути "внешняя" функция, описанная в классе <br>
    https://docs.python.org/3/library/functions.html#staticmethod <br>
    ***В программе используется для некоторых методов класса, которым не нужен объект на вход*** <br><br>
    
5. Конструкция `try except` - механизм языка Python для работы с ошибками <br>
    https://docs.python.org/3/tutorial/errors.html <br>
    ***В программе используется для корректной работы с вводом пользователя***

In [3]:
# P.S. Путь на 10 шагов - история о Рататуе